In [142]:
import serial
import time
import struct
import binascii
from ctypes import *
import pandas as pd

'20200514 15:42:30'

In [154]:
#讀取感測器資料v2
#port = "/dev/ttyS0"    # Raspberry Pi 3
port = "COM1"
ser = serial.Serial(port, baudrate = 921600, timeout = 1)
now = time.strftime("%Y%m%d%H%M")
saveFid = "acc_read_{}.txt".format(now)
data = b''
x=[]
y=[]
z=[]
try:
    while ser.isOpen:
        print(time.strftime('%Y%m%d %H:%M:%S'))
        with open(saveFid, "ab+") as file:
            aa=ser.read(96000).hex().split('4154')
            ff=[]
            for i in range(len(aa)):
                if len(aa[i]) == 3084:
                    ff.append(aa[i][4:-8])

                else:
                    pass
            tt=[]
            for j in range(len(ff)):
                n=8
                tt.append([ff[j][i:i+n] for i in range(0, len(ff[j]), n)])
            for i in range (len(tt)):
                for j in range(len(tt[i])):
                    if j % 3 == 0:
                        data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
                        x.append(data[0])
                    elif j % 3 == 1:
                        data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
                        y.append(data[0])
                    elif j % 3 == 2:
                        data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
                        z.append(data[0])
                    else:
                        pass
            
            file.write('X:'+x)
            file.write('Y:'+y)
            file.write('Z:'+z)
except KeyboardInterrupt:
    print(" -------Stop-------")
    ser.flushInput()
    ser.close()
    print(time.strftime('%Y%m%d %H:%M:%S'))
    

20200514 15:47:02
20200514 15:47:03
20200514 15:47:04
20200514 15:47:05
20200514 15:47:06
20200514 15:47:07
20200514 15:47:08
20200514 15:47:09
20200514 15:47:10
20200514 15:47:11
20200514 15:47:12
20200514 15:47:13
20200514 15:47:14
20200514 15:47:15
20200514 15:47:16
20200514 15:47:17
20200514 15:47:18
20200514 15:47:19
20200514 15:47:20
20200514 15:47:21
20200514 15:47:22
20200514 15:47:23
20200514 15:47:24
20200514 15:47:25
20200514 15:47:26
20200514 15:47:27
20200514 15:47:28
20200514 15:47:29
20200514 15:47:30
20200514 15:47:31
20200514 15:47:32
20200514 15:47:33
20200514 15:47:34
20200514 15:47:35
20200514 15:47:36
 -------Stop-------
20200514 15:47:37


In [155]:
#處理二進位資料
with open('acc_read_202005141547.txt','rb') as f:
    hexdata = f.read()
cc=hexdata
ff=[]
for i in range(len(cc)):
    if len(cc[i]) == 3084:
        ff.append(cc[i][4:-8])
        
    else:
        pass
tt=[]
for j in range(len(ff)):
    n=8
    tt.append([ff[j][i:i+n] for i in range(0, len(ff[j]), n)])

In [156]:
#測試
test_data=tt[0][5]
test_data
binary_string = binascii.unhexlify(test_data)
y=struct.unpack_from('f',binary_string)
y[0]

0.69183349609375

In [157]:
#資料轉換
x=[]
y=[]
z=[]
for i in range (len(tt)):
    for j in range(len(tt[i])):
        if j % 3 == 0:
            data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
            x.append(data[0])
        elif j % 3 == 1:
            data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
            y.append(data[0])
        elif j % 3 == 2:
            data = struct.unpack_from('f',binascii.unhexlify(tt[i][j]))
            z.append(data[0])
        else:
            pass
data_get = pd.DataFrame()
data_get['x']=pd.Series(x)
data_get['y']=pd.Series(y)
data_get['z']=pd.Series(z)
data_get.to_csv(str(now)+'.csv')